In [1]:
import dotenv

dotenv.load_dotenv()

True

In [2]:
import hopsworks

project = hopsworks.login(project="dbillLab1")
fs = project.get_feature_store()

/Users/martinbravodiaz/miniconda3/envs/imdb/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2025-01-04 08:41:11,282 INFO: Initializing external client
2025-01-04 08:41:11,283 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-04 08:41:12,816 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1169559


In [7]:
imdb_movie_fg = fs.get_feature_group(
    "tmdb_movies", 
    version=1
)
df = imdb_movie_fg.read()

df.head()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.11s) 


,id,title,vote_average,vote_count,status,release_date,revenue,runtime,budget,imdb_id,...,cast,director,director_of_photography,writers,producers,music_composer,imdb_rating,imdb_votes,poster_path,timestamp
0,106546,Moscow Heat,2.0,12.0,Released,2004-11-24,840201.0,89.0,0.0,tt0369738,...,"Joanna Pacula, Evgeniy Berezovskiy, Jeff Celen...",Jeff Celentano,John B. Aronson,"Alexander Nevsky, Alexander Izotov, Robert Madrid","Alexander Nevsky, Alexander Izotov",Richard John Baker,2.5,2438.0,/7WwaL21mcewnQTVAz3zOStDB4M1.jpg,2025-01-04 00:00:00+00:00
1,5526,A Killing Affair,5.2,4.0,Released,1977-09-21,0.0,100.0,0.0,tt0076266,...,"Todd Bridges, Sari Price, Dean Stockwell, Robe...",Richard C. Sarafian,Al Francis,E. Arthur Kean,"James H. Brown, David Gerber",Richard Shores,6.1,125.0,/hbb3Y27us6gx9nBpEOcMUcvh7qB.jpg,2025-01-04 00:00:00+00:00
2,194432,Top Secret,6.0,1.0,Released,1952-11-10,0.0,93.0,0.0,tt0045244,...,"Reed De Rouen, Henry Hewitt, Michael Medwin, W...",Mario Zampi,Stanley Pavey,"Michael Pertwee, Jack Davies",Mario Zampi,Stanley Black,6.3,201.0,/ss0h7JDyYMPYR2cP5fSlmpqQGD7.jpg,2025-01-04 00:00:00+00:00
3,319073,How to Change the World,7.1,33.0,Released,2014-12-12,0.0,110.0,0.0,tt4144504,...,"Robert Hunter, Bobbi Hunter, David Garrick, Pa...",Jerry Rothwell,Ben Lichty,Jerry Rothwell,"Bous De Jong, Stewart Le Marechal, Lizzie Fran...",Lesley Barber,7.5,1313.0,/994hnuZUbkAkpCZMaqasMw2cWvj.jpg,2025-01-04 00:00:00+00:00
4,68849,The Man Who Could Work Miracles,5.6,26.0,Released,1936-07-23,0.0,82.0,0.0,tt0029201,...,"George Zucco, Ernest Thesiger, Torin Thatcher,...",Lothar Mendes,Harold Rosson,"H.G. Wells, Lajos Biró",Alexander Korda,Mischa Spoliansky,6.9,1645.0,/yAfQRixZWBzQhJ7vwgCIyQhxNW9.jpg,2025-01-04 00:00:00+00:00


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("alanvourch/tmdb-movies-daily-updates")

In [ ]:
import os
import pandas as pd

# List files in the downloaded dataset directory
files = os.listdir(path)
print("Files in dataset directory:", files)

# Read the first file
updated_df = pd.read_csv(os.path.join(path, files[0]))

# Print the first 5 rows
updated_df.head()

In [ ]:
import datetime

def preprocess(df):
    # Drop rows with missing values
    df = df.dropna()
    
    # Only released movies
    df = df[df['status'] == 'Released']

    # Add timestamp column
    today = datetime.datetime.now().strftime("%Y-%m-%d")
    df['timestamp'] = pd.to_datetime(today)

    return df

updated_df = preprocess(updated_df)

In [23]:
import pandas as pd
from datetime import datetime

def update_df(df, updated_df):
    # Merge the two dataframes on 'id', keeping all rows from updated_df
    merged_df = pd.merge(updated_df, df, on='id', how='left', suffixes=('', '_old'))

    # Determine if the row is new, updated, or unchanged
    def determine_row(row):
        if pd.isna(row['timestamp_old']):  # New row
            return row['timestamp']
        elif row['timestamp'] > row['timestamp_old']:  # Updated row
            return row['timestamp']
        else:  # Unchanged row
            return row['timestamp_old']

    # Apply logic to set the correct timestamp
    merged_df['final_timestamp'] = merged_df.apply(determine_row, axis=1)

    # Drop unnecessary columns and rename back to the original format
    result_df = merged_df[['id', 'name', 'rating', 'final_timestamp']]
    result_df.rename(columns={'final_timestamp': 'timestamp'}, inplace=True)

    return result_df

# Original dataframe
df = pd.DataFrame({
    'id': [1, 2],
    'name': ['Movie A', 'Movie B'],
    'rating': [8.0, 7.5],
    'timestamp': [pd.Timestamp('2025-01-01 10:00:00'), pd.Timestamp('2025-01-01 10:00:00')]
})

# Updated dataframe
updated_df = pd.DataFrame({
    'id': [1, 2, 3],
    'name': ['Movie A', 'Movie B', 'Movie C'],
    'rating': [8.0, 6.0, 9.0],
    'timestamp': [pd.Timestamp('2025-01-02 15:00:00'), pd.Timestamp('2025-01-02 12:00:00'), pd.Timestamp('2025-01-02 15:00:00')]
})

# Expected dataframe
expected_df = pd.DataFrame({
    'id': [1, 2, 3],
    'name': ['Movie A', 'Movie B', 'Movie C'],
    'rating': [8.0, 6.0, 9.0],
    'timestamp': [pd.Timestamp('2025-01-01 10:00:00'), pd.Timestamp('2025-01-02 12:00:00'), pd.Timestamp('2025-01-02 15:00:00')]
})

# Update the original dataframe
result_df = update_df(df, updated_df)

# Test the result
assert result_df.equals(expected_df), "Dataframes are not equal"

print("Dataframe updated successfully:")
print(result_df)


In [ ]:
# Original dataframe
df = pd.DataFrame({
    'id': [1, 2],
    'name': ['Movie A', 'Movie B'],
    'rating': [8.0, 7.5],
    'timestamp': ['2025-01-01 10:00:00', '2025-01-01 10:00:00']
})


# Updated dataframe
updated_df = pd.DataFrame({
    'id': [1, 2, 3],
    'name': ['Movie A', 'Movie B', 'Movie C'],
    'rating': [8.0, 6.0, 9.0],
    'timestamp': ['2025-01-02 15:00:00', '2025-01-02 12:00:00', '2025-01-02 15:00:00']
})

expected_df = pd.DataFrame({
    'id': [1, 2, 3],
    'name': ['Movie A', 'Movie B', 'Movie C'],
    'rating': [8.0, 6.0, 9.0],
    'timestamp': ['2025-01-01 10:00:00', '2025-01-02 12:00:00', '2025-01-02 15:00:00']
})

# Update the original dataframe
result_df = update_df(df, updated_df)

assert result_df.equals(expected_df), "Dataframes are not equal"


KeyError: 'id'